In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [2]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [95]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [3]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com\noc\noc"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com/noc/noc'

In [99]:
%reset -f

# Examples

### MS SQL request

In [ ]:
sql_srt='SELECT TOP(5) * FROM USAGE_TYPE;'
with MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

In [ ]:
#reload(rb)

### Soap API OCS Portal

In [42]:
from zeep import Client
from zeep.wsse.username import UsernameToken

user = 'lab_api@roamability.com'
password = ''
api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
# api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl' # prod

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

ConnectionError: HTTPSConnectionPool(host='172.20.39.7', port=8585): Max retries exceeded with url: /ocsapi/roamability/api/roamability.wsdl (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001FFEA2925C0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение',))

In [44]:
%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

Choose connection (prod or lab):prod
You are connected to Reseller: Roamability Test RT.    
Available credit for the Reseller: 5999 USD.

The Reseller contains the following accounts:
 0    Roamability-Porto Test (accountID: 352332)
 1    Telzar Relay test (accountID: 352341)
 2    Roamability - SMPP TEST  (accountID: 352343)
 3    Telzar Relay (accountID: 352344)
 4    Roamability Test Sim-cards (accountID: 352349)
 5    Roamability MIFI Test (Denis) (accountID: 352415)
 6    RB Test hosted GGSN KR (accountID: 352424)
 7    DID's  (accountID: 352441)
 8    RB MiFi test account (accountID: 352446)
 9    soapApiTest (accountID: 352447)
 10   uCloudLink RB Test (accountID: 352448)
 11   Verint  (Roy's test) (accountID: 352471)
 12   Antoloy ( Roy's test) (accountID: 352472)
 13   Sure sample cards (accountID: 352479)
 14   Old Partner RT (accountID: 352527)
 15   Old Partner RT (accountID: 352528)
 16   aaa (accountID: 352529)
 17   Michael D test delete (accountID: 352531)
 18   Sigos 

In [643]:
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
client.service.assignSubscriberPrePaidPackage('209287', pp_pack_new, datetime.strptime(activation_date,'%Y-%m-%d'))
client.service.getSubscriberPrePaidPackages('1316409', 1)

{
    'result': {
        'code': '1',
        'description': 'Success',
        'errorUUID': None
    },
    'prePaidPackages': {
        'prePaidPackages': [
            {
                'prePaidPackageId': 8418,
                'subscriberId': 0,
                'locationZone': {
                    'locationZoneId': 17,
                    'locationZoneName': 'JoyTel Turkey',
                    'timeZoneName': 'Israel Standard Time',
                    'timeZoneUtcOffset': 2
                },
                'destinationZone': {
                    'destinationZoneId': 13,
                    'destinationZoneName': 'JoyTel Turkey'
                },
                'priority': 1,
                'assignedDate': datetime.datetime(2019, 3, 31, 16, 24, 46, 247000, tzinfo=<FixedOffset '+02:00'>),
                'activationDate': datetime.datetime(2019, 3, 31, 17, 24, 46, tzinfo=<FixedOffset '+03:00'>),
                'expirationDate': datetime.datetime(2019, 5, 1, 17, 24, 46, tzi

### Send MSU

In [508]:
from bs4 import BeautifulSoup
import roamability as rb
#reload(rb)

switch_01_payload = '24027000001f0d00010000bfff0100000000000010554f212dbbad49a1caf2573e1a626139'
switch_02_payload = '24027000001F0D00010000BFFF0100000000000040a740AD174703383798A7A6545BB36491'

#515030190000000
#1d46acf860780
add_imsi_03 = '3C02700000370d00210000bfff0100000000000040939360c23f417f147557b6940c91ba8b7e54be1c8c950a6ed1235ea8e3d70e9c2d7c34876abd68ec'


ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228',
       'porto_seguro':'550549900000',
       'rusec_rus':'79028710069',
       'rusec_int':'417999880000024',
       'multi_byte_sponsor':'85263347864',
       'smart1':'639180009880',
       'smart2':'639180009881',
       'smart3':'639180009882',
       'smart4':'639180009883',
       'telzar':'972559900040',
       'x2one':'972553316228',
       'cellact':'972557016315',
       'netmore':'46731726312',
       'p4_naka_01':'48790998145',
       'p4_naka_02':'48790998146',
       'p4_naka_03':'48790998147',
       'partner_naka_01':'97254120634',
       'partner_naka_02':'97254120635',
       'partner_naka_03':'97254120636',
       'partner_naka_04':'97254120637',}

rb.greetings_func()
#rb.print_imsi_prof(ogt, imsi, msc, 'as_resp') #'as_mo'
#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml') #250359000007598 79028710043
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi, node),'xml') # 0 - vlr; 16 - mme
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, rb.get_info),'xml')
#soup = BeautifulSoup(rb.cl(ogt, dgt, imsi, d_ssn=7),'xml')
#soup

Greetings from Roamability!!! 



In [103]:
get_info    = "24027000001F0D00010000BFFF01000000000000102F978372C730834EF445172BD26B8C8F"
get_info_mo = "24027000001F0D00210000BFFF01000000000000102F978372C730834EF445172BD26B8C8F"

In [68]:
rb.greetings_func()
rb.print_imsi_prof('66893773228', '520151101262057', '66893770000', 'as_mo') #'as_mo' 'as_resp'

Greetings from Roamability!!! 

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>T_ABORT received</error>
<src_gt/>
</response>> 

No info in response.


In [263]:
def decode_payload_response(resp):
    s = resp.upper()
    #imsi_list = re.findall('(0\d08)(\d{16})', s)
    #imsi_list = re.findall('(0\d08\d9\d{14})', s)
    imsi_list = re.findall('(0\d08)(\d9\d{14})', s)
    print(imsi_list)
    print('This SIM card contains the following IMSIs:')
    for slot, imsi in imsi_list:
        print(f"Slot {slot[1]} {''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])}")
    app_ver = re.findall('FFFFFFF000(\d{6})', s)
    if app_ver:
        print(f'\nApplet ver.: {int(app_ver[0][:2])}.{int(app_ver[0][2:4])}.{int(app_ver[0][4:6])}')
    mccmnc_extract = re.findall('FFFFFFF000\d{6}(.{6})', s)
    if mccmnc_extract:
        mccmnc = ''.join([mccmnc_extract[0][i] for i in [1, 0, 3, 5, 4]])
        print(f'MCCMNC: {mccmnc}')

s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF0000107020000000100033A75350738900608000849528100020000600108495210690210139702082906601004901438'
decode_payload_response(s)

[('0008', '4952810002000060'), ('0108', '4952106902101397'), ('0208', '2906601004901438')]
This SIM card contains the following IMSIs:
Slot 0 425180020000006
Slot 1 425019620013179
Slot 2 260060140094183

Applet ver.: 1.7.2
MCCMNC: 00000


In [69]:
reload(rb)
s = '00350abfff0100000000000000fffffff00001070052f0200000003a155608057823070008495210693100112901082906601094001129'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070232F4010000003A95940448388403000849528100020000500108495210690210620102082906601004803854'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF0000107020000000100033A75350738900608000849528100020000600108495210690210139702082906601004901438'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070025F0510000003A55130828920808000859025111106202550108495210690210749802082906601004114330'
rb.decode_payload_response(s)

This SIM card contains the following IMSIs:
Slot 0 520151101262055
Slot 1 425019620014789
Slot 2 260060140113403

Applet ver.: 1.7.0
MCCMNC: 52015


In [895]:
import http
import re
from bs4 import BeautifulSoup
import sys

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type": "text/xml"})
    resp = client.getresponse()
    return resp.read()

def sri4sm(ogt, msisdn, dgt):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <sccp_np>7</sccp_np>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, dgt, msisdn, ogt)

    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp
soup = BeautifulSoup(sri4sm('97254120634', '972541014729', '972549612513794'),'xml') #250359000007598 79028710043
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [288]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
any_tags  = ['#reports_bss_usage_mop']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\UsageSponsorReport.txt
----------------------------------------------------------------------------------------------------
#reports_bss_usage_mop
#reports_tariffs_mop
------------------------------
Отчеты по Cost по Споносрам

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_Customer.txt
----------------------------------------------------------------------------------------------------
#reports_bss_usage_mop
------------------------------
Отчет по кастомеру, usage_type, month

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt
------------------------------------------------------------

### Search in ALARMS by folder names

In [584]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
#          r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('NLDPT')!=-1]

c:\W_DATA_ROAM\ALARMS\PS_Netherlands_NLDPT_180508_#1565799


### Search in Allowed Lists and others

In [425]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Sat_Jun_29_2019.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Mar_20_2019.csv'
bss_usage = 'bss_usage_2019_06.csv'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*':'S1', r'[Pp]4[\s\w-]*':'S2'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads, dmi_netpfx))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [426]:
sponsor = 'p4' # Use regular expressions if it's needed
tadig = 'SLV'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA
2552,P4_All,SLVTS,NaN,NaN,NaN,NaN
912,P4_All Rated,SLVTS,NaN,NaN,NaN,NaN
3753,P4_Dialoq_ALL,SLVTS,NaN,NaN,NaN,NaN


In [27]:
df_np[(df_np.Operator.str.startswith('INDAT',na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC


In [12]:
plmn = 'CYPSC'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
129,P4_All,CYPSC
3423,P4_Combined,CYPSC
3658,P4_Dialoq_ALL,CYPSC
3863,P4_RUSEC_Com,CYPSC
3261,P4_STI_Comb,CYPSC
231,P4_WeStream,CYPSC


,Operator,Prefix,Number/Range,SSN,MCC,MNC
1329,CYPSC,35796,1,149,280,10
3642,CYPSC,3579696700,0,7,280,10
4983,CYPSC,35796,1,7,280,10
8877,CYPSC,3579696700,0,149,280,10
10135,CYPSC,35796967000,0,1,280,10


In [379]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('6688888',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC


### Hex to Dec conversion

In [130]:
hex_str = '1172'
dec_str = '2505'
dec_int = 1962
bin_str = '1010'

# Hex to dec
print('Hex to dec')
print(int(hex_str,16))

# Dec to hex
print('Dec to hex')
print(hex(dec_int).upper())
print('%x'.upper() % dec_int)

# Dec to bin
print('Dec to bin')
print(bin(dec_int)[2:].zfill(8))

# Bin to dec
print('Bin to dec')
print(int(bin_str,2))

# Hex to bin
print('Hex to bin')
print(bin(int(hex_str, 16))[2:].zfill(8))
print(bin(0xABC123EFFF))
f'{0xAB:0>16b}'

Hex to dec
4466
Dec to hex
0X7AA
7AA
Dec to bin
11110101010
Bin to dec
10
Hex to bin
1000101110010
0b1010101111000001001000111110111111111111


'0000000010101011'

In [188]:
f'{bin(2)} - {bin(61)} - {bin(6)}'
#2.61.6

'0b10 - 0b111101 - 0b110'

In [186]:
print(bin(10915))
a = '10101010100011'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
5 - 84 - 3


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [86]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2019-01-24 05:00:45,431 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.978321
8 2019-01-23 05:00:47,552 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.941926
16 2019-01-22 05:00:49,749 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.060281
24 2019-01-21 05:00:49,224 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.623832
32 2019-01-20 05:00:47,066 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.239989
40 2019-01-19 05:00:45,864 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.761820
48 2019-01-18 05:00:46,195 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.864928


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [ ]:
import requests
#payload = {'key1':'value1','key2':'value2'}
#r = requests.get('http://192.168.1.44:80/climate;sensor=1&readattempt=10&temperature=nan&humidity=nan')
r = requests.get('http://192.168.1.44:80/climate;sensor=1&readattempt=10&temperature=10.0&humidity=10.0')
#r = requests.get('http://192.168.1.53:80/connected;IpAddr=192.168.1.63')
print('Status: ',r.status_code,'\n',r.text)
r.close()

In [160]:
import smpplib
import smpplib.gsm
client = None
#parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'''
Test!
''')

client = smpplib.client.Client('176.10.126.173', 2770)
client.connect()
client.bind_transceiver(system_id='test', password='test')
for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
        #source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
        source_addr='97254120000',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        #dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
        destination_addr='447781505034',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.unbind()
client.disconnect()

PDU SQN: 2


In [39]:
import requests
ip_addr = '192.168.1.52'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

200
"OK"
200
"OK"
200
"OK"
200
"OK"


In [8]:
import cx_Oracle
con = cx_Oracle.connect('DMI/dd607605ce341@DMI_TEST')
print(con.version)
cur = con.cursor()
cur.execute('SELECT COUNT(*) FROM S_IMSI')
for result in cur:
    print(result)
cur.close()
con.close()

(293,)


In [586]:
260060169999999 - 260060169000000 + 1

1000000

In [28]:
%reset -f